In [1]:
import pandas as pd
import numpy as np

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
window_size = 60
Npi = 100
bandwidth = 0.1
all_prices = []
for ticker in tickers:
    file_path = f"{folder}{ticker}.csv"
    df = pd.read_csv(file_path, parse_dates=['Date'])
    price_col = 'Adjusted Close' if 'Adjusted Close' in df.columns else 'Adj Close'
    df = df[['Date', price_col]].rename(columns={price_col: ticker})
    all_prices.append(df)

data = all_prices[0]
for df in all_prices[1:]:
    data = data.merge(df, on='Date', how='inner')
data = data.set_index('Date')
data = data.loc["2011-01-01":"2020-12-31"]
log_returns = np.log(data / data.shift(1)).dropna()

def create_sequences(log_returns, window):
    sequences = []
    for i in range(len(log_returns) - window):
        seq = log_returns.iloc[i:i+window].values
        sequences.append(seq)
    return np.array(sequences)

data_series = create_sequences(log_returns, window_size)

def quartic_kernel(x, h):
    norm = np.linalg.norm(x) / h
    return ((1 - norm**2)**2) / h if norm < 1 else 0.0

def Fi(t, xti, x, xtip1, ti, tip1):
    dt1 = tip1 - t
    dt2 = tip1 - ti
    term1 = -np.linalg.norm(xtip1 - x)**2 / (2 * dt1)
    term2 = np.linalg.norm(xtip1 - xti)**2 / (2 * dt2)
    return np.exp(term1 + term2)

def estimate_drift(t, x, xi, data_series, ti, tip1, h):
    num = np.zeros_like(x)
    denom = 0.0
    for sample in data_series:
        if tip1 >= sample.shape[0]:
            continue
        xti = sample[ti]
        xtip1 = sample[tip1]
        Fi_val = Fi(t, xti, x, xtip1, ti, tip1)
        weight = Fi_val
        for j in range(ti):
            weight *= quartic_kernel(xi[j] - sample[j], h)
        num += weight * (xtip1 - x)
        denom += weight
    return (1 / (tip1 - t)) * num / (denom + 1e-8)

def sbts_simulation(data_series, Npi=100, h=0.1):
    M, N, d = data_series.shape
    x = [np.zeros(d)]
    for i in range(N - 1):
        y = x[i].copy()
        for k in range(Npi - 1):
            tpi_k = i + k / Npi
            drift = estimate_drift(tpi_k, y, np.array(x), data_series, ti=i, tip1=i+1, h=h)
            eps = np.random.randn(d)
            y = y + (1 / Npi) * drift + (1 / np.sqrt(Npi)) * eps
        x.append(y)
    return np.array(x)

simulated = sbts_simulation(data_series, Npi=Npi, h=bandwidth)
df_sim = pd.DataFrame(simulated, columns=tickers)
print(df_sim)

/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_64644/594588895.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_64644/594588895.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_64644/594588895.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681d

         AAPL      MSFT       GOOG       AMZN       NFLX
0    0.000000  0.000000   0.000000   0.000000   0.000000
1    0.187889  0.147329  -0.127145   0.072663  -0.069450
2   -0.170019  0.041800   0.105529  -0.066686  -0.013116
3   -1.232076 -2.014651   0.614796  -1.154960   1.151696
4   -1.543696 -1.334735   2.329717  -0.102424   2.470755
5   -0.780871 -1.688138   4.303064   0.329228   2.442110
6   -3.884575 -0.483655   5.216083   0.160892   2.922536
7   -3.148595 -1.180695   6.707892   1.238117   2.850359
8   -3.879745 -1.008958   6.749282   0.272056   2.729758
9   -4.270371 -1.823235   6.524562  -1.203504   4.127909
10  -4.509090 -2.687446   6.705262  -1.061636   4.955333
11  -5.138124 -3.632084   7.392638   0.441954   5.506422
12  -5.722220 -3.365496   7.868338   1.177868   5.102685
13  -7.067911 -2.639136   7.804259   1.795568   5.195517
14  -7.920576 -4.788484   7.037137   4.385306   5.055213
15  -8.405240 -6.755781   7.755923   3.514723   4.000047
16  -9.484020 -5.050774   8.527

In [4]:
returns = df_sim.diff().dropna()

mu = returns.mean().values
cov = returns.cov().values
inv_cov = np.linalg.inv(cov)
ones = np.ones(len(mu))

w_sharpe = inv_cov @ mu
w_sharpe /= np.sum(w_sharpe)

portfolios = pd.DataFrame([w_sharpe],
                          columns=df_sim.columns,
                          index=['MaxSharpe'])
print(portfolios)

               AAPL      MSFT      GOOG      AMZN      NFLX
MaxSharpe -0.434541 -0.172563  0.467519 -0.470178  1.609763


In [5]:
import os

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
start_date = '2021-01-01'
end_date = '2021-03-01'
initial_investment = 100000
weights = np.array([-0.434541, -0.172563, 0.467519, -0.470178, 1.609763])

close_prices = pd.DataFrame()

for ticker in tickers:
    filepath = os.path.join(folder, f"{ticker}.csv")
    df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
    close_prices[ticker] = df['Adjusted Close']

close_prices = close_prices.loc[start_date:end_date]

start_prices = close_prices.iloc[0]
end_prices = close_prices.iloc[-1]

shares_bought = (initial_investment * weights) / start_prices

final_value = np.sum(shares_bought * end_prices)

print(f"Initial investment: ${initial_investment:,.2f}")
print(f"Final portfolio value after 2 months: ${final_value:,.2f}")
print(f"Return: {(final_value / initial_investment - 1) * 100:.2f}%")

Initial investment: $100,000.00
Final portfolio value after 2 months: $139,804.27
Return: 39.80%


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_64644/4225628423.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_64644/4225628423.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_64644/4225628423.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['